<a href="https://colab.research.google.com/github/Angorith/acorn_python/blob/main/0719_49%EC%9D%BC%EC%B0%A8_UCI_HAR_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cp drive/MyDrive/UCI_HAR_Dataset.zip .

In [ ]:
!mkdir UCI_HAR_Dataset

In [ ]:
!unzip UCI_HAR_Dataset.zip -d UCI_HAR_Dataset/

Archive:  UCI_HAR_Dataset.zip
  inflating: UCI_HAR_Dataset/activity_labels.txt  
  inflating: UCI_HAR_Dataset/features.txt  
  inflating: UCI_HAR_Dataset/X_test.txt  
  inflating: UCI_HAR_Dataset/X_train.txt  
  inflating: UCI_HAR_Dataset/y_test.txt  
  inflating: UCI_HAR_Dataset/y_train.txt  


In [ ]:
#!mv *.txt UCI_HAR_Dataset/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
feature_name_df = pd.read_csv('UCI_HAR_Dataset/features.txt', 
                              sep ='\s+', 
                              header=None , 
                              names=['column_index', 'column_name'])

In [ ]:
feature_name_df

,column_index,column_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [ ]:
feature_name_df.groupby('column_name').cumcount()

0      0
1      0
2      0
3      0
4      0
      ..
556    0
557    0
558    0
559    0
560    0
Length: 561, dtype: int64

In [ ]:
feature_name_df.groupby('column_name').cumcount().sum()

126

In [ ]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data= old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1])
                                                                                               if x[1]>0 else x[0], axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [ ]:
new_feature_name_df = get_new_feature_name_df(feature_name_df)

In [ ]:
new_feature_name_df.count()

column_index    561
column_name     561
dup_cnt         561
dtype: int64

In [ ]:
new_feature_name_df.groupby('dup_cnt').cumcount()

0        0
1        1
2        2
3        3
4        4
      ... 
556    472
557    473
558    474
559    475
560    476
Length: 561, dtype: int64

In [ ]:
def get_human_dataset():
    #각 데이터 파일은 공백으로 분리되어 있어 read_csv에서 공백 문자를 sep로 할당
    feature_name_df = pd.read_csv('UCI_HAR_Dataset/features.txt', sep ='\s+', header=None , names=['column_index', 'column_name'])
    
    #중복된 피처명을 수정하는 get_new_feature_name_df를 이용, 신규 피처데이터프레임 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    #데이터프레임에 피처명을 컬럼으로 부여하기 위해 리스트객체로 변환
    feature_name = new_feature_name_df.iloc[:, 1].values.tolist()
    
    #학습 피처 데이터 세트와 테스트 피처 데이터를 dataframe으로 로딩, 컬럼명은 feature_name적용
    X_train = pd.read_csv('UCI_HAR_Dataset/X_train.txt', sep ='\s+', names = feature_name)
    X_test = pd.read_csv('UCI_HAR_Dataset/X_test.txt', sep ='\s+', names = feature_name)

    y_train = pd.read_csv('UCI_HAR_Dataset/y_train.txt', sep ='\s+', names = ['action'])
    y_test = pd.read_csv('UCI_HAR_Dataset/y_test.txt', sep ='\s+', names = ['action'])

    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_human_dataset()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7352, 561), (2947, 561), (7352, 1), (2947, 1))

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, tBodyAcc-mean()-X to angle(Z,gravityMean)
dtypes: float64(561)
memory usage: 31.5 MB


In [ ]:
dt_clf = DecisionTreeClassifier(random_state=156)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test, pred)
print('결정 트리 예측 정확도 : {0:.4f}'.format(accuracy))
print('DecisionTreeClassifier 기본 하이퍼 파라미터 : \n', dt_clf.get_params())

결정 트리 예측 정확도 : 0.8548
DecisionTreeClassifier 기본 하이퍼 파라미터 : 
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth' : [6,8,10,12,16,20,24]}
grid_cv = GridSearchCV(dt_clf, param_grid = params, scoring = 'accuracy', cv=5, verbose=1)
grid_cv.fit(X_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치 : {0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터 : ', grid_cv.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
GridSearchCV 최고 평균 정확도 수치 : 0.8513
GridSearchCV 최적 하이퍼 파라미터 :  {'max_depth': 16}
